## Notebook Imports

In [1]:
import pandas as pd
import numpy as np

## Constants

In [2]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'
VOCAB_SIZE = 2500

## Read and Load Features from .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
sparse_train_data[-5:]

array([[5795, 2056,    0,    1],
       [5795, 2059,    0,    1],
       [5795, 2284,    0,    1],
       [5795, 2359,    0,    1],
       [5795, 2442,    0,    1]])

In [7]:
print('No of rows in training file', sparse_train_data.shape[0])
print('No of rows in test file', sparse_test_data.shape[0])

No of rows in training file 258328
No of rows in test file 117640


In [8]:
print('No of emails in training file', np.unique(sparse_train_data[:, 0]).size)

No of emails in training file 4015


In [9]:
print('No of emails in test file', np.unique(sparse_test_data[:, 0]).size)

No of emails in test file 1724


### How to Create an Empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:, 0])  # Select first column
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [15]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [16]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
sparse_train_data[10][1]

31

## Create a Full Matrix from a Sparse Matrix

In [19]:
def make_full_matrix(sparse_matrix, no_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix -- numpy array
    no_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of the occurrence of word in the sparse matrix. Default: 4th column
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, no_words))
    doc_id_names = np.unique(sparse_matrix[:, doc_idx])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_no = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_no, 'DOC_ID'] = doc_no
        full_matrix.at[doc_no, 'CATEGORY'] = label
        full_matrix.at[doc_no, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [20]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 10.3 s


In [22]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,1,1,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


## Training the Naive Bayes Model

### Calculating the Probability of Spam

In [44]:
# Calculate the probability of spam - the percent of spam messages in the training dataset.
total_no_emails = full_train_data.shape[0]
no_spam_emails = full_train_data[full_train_data['CATEGORY'] == 1].shape[0]
prob_spam = no_spam_emails / total_no_emails
print('Probability of spam is', prob_spam)

Probability of spam is 0.31133250311332505


## Total Number of Words / Tokens

In [45]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4015,)

In [47]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    182
4     43
dtype: int64

In [48]:
total_wc = email_lengths.sum()
total_wc

430962

## Number of Tokens in Spam & Ham Emails

In [55]:
spam_lengths = email_lengths[full_train_data['CATEGORY'] == 1]
spam_wc = spam_lengths.sum()

ham_lengths = email_lengths[full_train_data['CATEGORY'] == 0]
ham_wc = ham_lengths.sum()

print('Total no of words in spam emails:', spam_wc)
print('Total no of words in ham emails:', ham_wc)

Total no of words in spam emails: 178667
Total no of words in ham emails: 252295


In [57]:
spam_wc + ham_wc - total_wc  # Check sum

0

In [60]:
print('Average no of words in spam emails: {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average no of words in ham emails: {:.3f}'.format(ham_wc / ham_lengths.shape[0]))

Average no of words in spam emails: 143
Average no of words in ham emails: 91.246
